In [40]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install keras
!pip install tensorflow

In [8]:
prices = pd.read_csv('/content/FB-stock-prices.csv')

In [57]:
prices.head()

,Date,Price
0,1/4/2021,269
1,1/5/2021,271
2,1/6/2021,263
3,1/7/2021,269
4,1/8/2021,268


In [4]:
prices.dtypes

,0
Date,object
Price,int64


In [6]:
plt.figure(figsize=20,10)
plt.plot(prices['Date'], prices['Close'])
plt.show()

TypeError: Value after * must be an iterable, not int

In [7]:
prices.count()

,0
Date,251
Price,251


In [13]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_prices=scaler.fit_transform(prices[["Price"]].values)

In [15]:
# print(scaled_prices)

In [16]:
total_size=len(scaled_prices)

In [17]:
test_size=50
train_size=total_size-test_size
print(total_size)
print(test_size)
print(train_size)

251
50
201


In [21]:
total_size = len(scaled_prices)
test_size = 50
train_size = total_size - test_size
print(total_size)
print(test_size)
print(train_size)

# Use the calculated train_size (which is an integer) for slicing
train_prices = scaled_prices[0:train_size]  # Changed variable name to train_prices
test_prices = scaled_prices[train_size:total_size] # Use total_size instead of len(scaled_prices) for consistency

251
50
201


In [22]:
print(train_prices.shape)

(201, 1)


In [29]:
def create_rnn_dataset(data,lookback=1):
  data_x,data_y=[]
  for i in range(len(data)-lookback-1):
    a=data[i:(i+lookback),0]
    data_x.append(a)
    data_y.append(data[i+lookback,0])
  return np.array(data_x),np.array(data_y)

In [34]:
def create_rnn_dataset(data, lookback=1):
  # Initialize data_x and data_y as separate empty lists
  data_x, data_y = [], []
  for i in range(len(data) - lookback - 1):
    a = data[i:(i + lookback), 0]
    data_x.append(a)
    data_y.append(data[i + lookback, 0])
  return np.array(data_x), np.array(data_y)

In [42]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense  # Change simpleRNN to SimpleRNN
import tensorflow as tf

In [44]:
price_model=Sequential()
price_model.add(SimpleRNN(32,input_shape=(1,lookback)))
price_model.add(Dense(1))
price_model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse']) # Changed 'metrices' to 'metrics'

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
price_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)             │ (None, 32)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,121 (4.38 KB)

 Trainable params: 1,121 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
lookback = 1  # Define the lookback period, for example 1

# Generate train_x and train_y using your create_rnn_dataset function
train_x, train_y = create_rnn_dataset(train_prices, lookback)

# Reshape train_x to match the expected input shape of the RNN
train_x = train_x.reshape(train_x.shape[0], 1, train_x.shape[1])

# Now you can fit the model:
price_model.fit(train_x, train_y, epochs=10, batch_size=1)

Epoch 1/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0268 - mse: 0.0268
Epoch 2/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0249 - mse: 0.0249
Epoch 3/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0256 - mse: 0.0256
Epoch 4/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0362 - mse: 0.0362
Epoch 5/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0311 - mse: 0.0311
Epoch 6/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0301 - mse: 0.0301
Epoch 7/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0246 - mse: 0.0246
Epoch 8/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0254 - mse: 0.0254
Epoch 9/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0280 - mse: 0.0280
Epoch 10/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0272 - mse: 0.0272


In [50]:
test_x, test_y = create_rnn_dataset(test_prices, lookback)
test_x = test_x.reshape(test_x.shape[0], 1, test_x.shape[1])

In [51]:
price_model.evaluate(test_x, test_y,verbose=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0424 - mse: 0.0424  


[0.04075134918093681, 0.04075134918093681]

In [52]:
predict_on_test=price_model.predict(test_x)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step


In [53]:
print(predict_on_test)

[[ 0.58972037]
 [ 0.6196593 ]
 [ 0.09979033]
 [ 0.22373073]
 [-0.18029355]
 [-0.3045846 ]
 [-0.14916924]
 [ 0.0687203 ]
 [ 0.2546067 ]
 [ 0.19280633]
 [ 0.31619415]
 [ 0.43858147]
 [ 0.58972037]
 [ 0.5295411 ]
 [ 0.40809456]
 [ 0.19280633]
 [ 0.19280633]
 [ 0.58972037]
 [ 0.79693794]
 [ 0.6494925 ]
 [ 0.58972037]
 [ 0.5295411 ]
 [ 0.708826  ]
 [ 0.58972037]
 [ 0.46898782]
 [ 0.58972037]
 [ 0.3468963 ]
 [ 0.49930915]
 [ 0.0687203 ]
 [-0.33558097]
 [-0.36653706]
 [-0.45911458]
 [-0.11803423]
 [ 0.03762606]
 [ 0.2854294 ]
 [ 0.2546067 ]
 [ 0.2546067 ]
 [ 0.37753126]
 [ 0.37753126]
 [ 0.6196593 ]
 [ 0.40809456]
 [ 0.37753126]
 [ 0.09979033]
 [ 0.37753126]
 [ 0.2546067 ]
 [ 0.40809456]
 [ 0.7383188 ]
 [ 0.7383188 ]]


In [55]:
predicted_values=scaler.inverse_transform(predict_on_test)

In [56]:
print(predicted_values)

[[341.68268]
 [342.72562]
 [324.6157 ]
 [328.93323]
 [314.85883]
 [310.52908]
 [315.94305]
 [323.53336]
 [330.0088 ]
 [327.85596]
 [332.15424]
 [336.41766]
 [341.68268]
 [339.58627]
 [335.35562]
 [327.85596]
 [327.85596]
 [341.68268]
 [348.9012 ]
 [343.76486]
 [341.68268]
 [339.58627]
 [345.8318 ]
 [341.68268]
 [337.47687]
 [341.68268]
 [333.22375]
 [338.53314]
 [323.53336]
 [309.4493 ]
 [308.37094]
 [305.14594]
 [317.02765]
 [322.45016]
 [331.08252]
 [330.0088 ]
 [330.0088 ]
 [334.29095]
 [334.29095]
 [342.72562]
 [335.35562]
 [334.29095]
 [324.6157 ]
 [334.29095]
 [330.0088 ]
 [335.35562]
 [346.85916]
 [346.85916]]


predictions


In [59]:
prev_prices=np.array([[397.99, 371.73, 329.47, 319.76, 386.20,
 390.23, 383.76, 324.57, 388.63, 377.40,
 326.01, 322.49, 372.58, 388.32, 343.30,
 325.84, 315.36, 356.06, 363.19, 312.64,
 383.17, 387.51, 387.91, 382.71, 398.23]])

# Reshape prev_prices to have a single feature per sample
prev_prices = prev_prices.reshape(-1, 1)

# Use the original scaler ('scaler', not 'scalar')
scaled_prices = scaler.transform(prev_prices)

# Reshape scaled_prices for the RNN model input
model_input = scaled_prices.reshape(scaled_prices.shape[0], 1, 1)

pred = price_model.predict(model_input)
stock_pred = scaler.inverse_transform(pred)
print(stock_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[392.5048 ]
 [371.54688]
 [329.43896]
 [318.93692]
 [383.6617 ]
 [386.78848]
 [381.7159 ]
 [324.1504 ]
 [385.56006]
 [376.45798]
 [325.70782]
 [321.89746]
 [372.29633]
 [385.3201 ]
 [344.0759 ]
 [325.52408]
 [314.1652 ]
 [356.94055]
 [363.76663]
 [311.2208 ]
 [381.23944]
 [384.69   ]
 [385.0017 ]
 [380.86633]
 [392.67523]]
